In [ ]:
import numpy as np #for numerical calculations
import matplotlib.pyplot as plt 
import pandas as pd 
import yfinance as yf 
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler

In [ ]:
ticker = 'GOOG'

start  = '2012-01-11'
end    = '2023-07-10'


data  = yf.download(ticker,start=start,end=end)


In [ ]:
#Adding indicators to our data
period =15
data['RSI'] = ta.rsi(data.Close,length=period)

data['EMAF'] = ta.ema(data.Close,length=20)

data['EMAM'] = ta.ema(data.Close,length=100)

data['EMAS'] = ta.ema(data.Close,length=150)

#Create the features 
data['Target'] = data['Adj Close'].pct_change()
data['Target'] = data.Target.shift(-1)

In [ ]:
#create the target class
#The data comes after a list comprehension of the pct change

data['TargetClass'] = [ 1 if data.Target[i] > 0 else 0 for i in range(len(data))]
print(data['TargetClass'])

data['TargeNextClose'] = data['Adj Close'].shift(-1)#We then move on to the next close


In [ ]:
data.reset_index(inplace=True)
data.drop(['Volume','Close','Date'],axis=1,inplace=True)
data_set = data.iloc[:,0:11]

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_set)
print(data_scaled)

In [ ]:
#Generating our X and y data

X = []
backcandles = 30 #Lookback on the last candles
#Extract the columns 
for j in range(8):
    X.append([])
    for i in range(backcandles,data_scaled.shape[0]):
        X[j].append(data_scaled[i-backcandles:i,j])
X = np.moveaxis(X,[0],[2])
#we are only taking the 30 days and last column for classification
X,yi = np.array(X),np.array(data_scaled[backcandles:,-1])
y = np.reshape(yi,len(yi),1)

In [ ]:
#Split into training and testing 
split = int(len(X) * 0.8)

x_train , y_train = X[:split] , y[:split]
x_test , y_test  = X[split:] , y[split:]

In [ ]:
import tensorflow as tf 
import keras 
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense,Dropout,LSTM,Input,Activation,concatenate
np.random.seed(10)


In [ ]:
def model(activation='linear'):
    #creation of the model and inputs are created 
    inputs_lstm = Input(shape=(backcandles,8),name='model_inputs')
    inputs = LSTM(150,name='first_layer')(inputs_lstm)#creates the model and initializes the shape 
    inputs = Dense(150,name='dense_layer')(inputs)
    output_layer = Activation(activation,name='outputs')(inputs)
    model = Model(inputs = inputs_lstm,outputs=output_layer)
    adam = optimizers.Adam()
    model.compile(optimizer=adam,loss='mse')
    model.fit(x=x_train,y=y_train,batch_size=15,epochs=25,shuffle = True,validation_split=0.1)

    return model



In [ ]:
model = model()
y_pred = model.predict(X_test)
#evaluate the model predictions
for i in range(10):
    print(f'Predictions{y_pred[i]}, Actual:{y_test[i]}')

#plot the model results
plt.figure(figsize=(16,8))
plt.plot(y_test,color='blue',label='Test Data')
plt.plot(y_pred,color='green',label='Model Predictions')
plt.legend()
plt.show()

    